In [1]:
from datasets import load_dataset
from training_datasets.GLUEDataset import GLUEDataset

cola = load_dataset('glue','cola')
# print(cola)
# sst2 = load_dataset('glue','sst2')
# print(sst2)
mrpc = load_dataset('glue', 'mrpc')
# print(mrpc)
pt_cola = GLUEDataset([cola, mrpc], 'train')

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1052.26it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1275.12it/s]


In [2]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

# sampler = FewShotEpisodeSampler(pt_cola, kShot=2, nWay=4, shuffle=True)
# train_data_loader = data.DataLoader(
#     pt_cola,
#     batch_sampler=sampler,
#     num_workers=1,
# )

# data, targets = next(iter(train_data_loader))

# for i in range(5):
#     data, targets = next(iter(train_data_loader))
#     print(data)
#     print(targets)

train_protomaml_sampler = FewShotEpisodeBatchSampler(pt_cola, kShot=3, nWay=4, batchSize=8, shuffle=True)
train_protomaml_loader = data.DataLoader(
    pt_cola, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=1
)

# for i in range(1):
#     batch = next(iter(train_protomaml_loader))
#     for episode_i in range(len(batch[0])):
#         data, labels = batch[0][episode_i], batch[1][episode_i]
#         supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportLabels)

In [3]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [4]:
from training.models.ProtoFOMAML import ProtoFOMAML

pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
pfomaml.training_step(next(iter(train_protomaml_loader)), 0)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.

losses are tensor([1.1847, 1.0977, 1.0563, 0.9941, 1.1267, 1.1590],
       grad_fn=<NllLossBackward0>) and loss is 6.618457317352295
losses are tensor([1.1343, 1.0538, 1.0117, 0.9270, 1.0647, 1.1222],
       grad_fn=<NllLossBackward0>) and loss is 6.313683032989502
losses are tensor([1.0864, 1.0126, 0.9710, 0.8642, 1.0059, 1.0882],
       grad_fn=<NllLossBackward0>) and loss is 6.028391361236572
losses are tensor([1.0409, 0.9740, 0.9332, 0.8060, 0.9503, 1.0562],
       grad_fn=<NllLossBackward0>) and loss is 5.760585308074951
losses are tensor([0.9982, 0.9374, 0.8976, 0.7521, 0.8979, 1.0258],
       grad_fn=<NllLossBackward0>) and loss is 5.508958339691162
losses are tensor([1.0949, 1.2037, 1.1558, 1.1601, 1.0153, 1.0449],
       grad_fn=<NllLossBackward0>) and loss is 6.674637794494629
losses are tensor([1.0694, 1.1863, 1.0846, 1.1357, 0.9434, 0.9573],
       grad_fn=<NllLossBackward0>) and loss is 6.376493453979492
losses are tensor([1.0454, 1.1690, 1.0200, 1.1118, 0.8799, 0.8794],
 

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/core/module.py:410: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(
/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/core/module.py:410: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.pre

losses are tensor([1.0129, 1.0880, 1.1125, 1.0542, 1.0016, 1.0958],
       grad_fn=<NllLossBackward0>) and loss is 6.3650288581848145
losses are tensor([0.9235, 1.0033, 1.0416, 0.9752, 0.9315, 1.0136],
       grad_fn=<NllLossBackward0>) and loss is 5.88877010345459
losses are tensor([0.8561, 0.9346, 0.9851, 0.9105, 0.8745, 0.9487],
       grad_fn=<NllLossBackward0>) and loss is 5.509384632110596
losses are tensor([0.8023, 0.8771, 0.9389, 0.8567, 0.8273, 0.8990],
       grad_fn=<NllLossBackward0>) and loss is 5.201318740844727
losses are tensor([0.7582, 0.8306, 0.9013, 0.8131, 0.7872, 0.8574],
       grad_fn=<NllLossBackward0>) and loss is 4.94773006439209
losses are tensor([1.0255, 1.0405, 1.1153, 1.1687, 1.0918, 1.0126],
       grad_fn=<NllLossBackward0>) and loss is 6.4544501304626465
losses are tensor([0.9657, 0.9523, 1.0429, 1.0935, 0.9932, 0.9184],
       grad_fn=<NllLossBackward0>) and loss is 5.96586799621582
losses are tensor([0.9192, 0.8842, 0.9886, 1.0349, 0.9173, 0.8459],
  

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/core/module.py:410: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(
/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/core/module.py:410: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.pre

...

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/core/module.py:410: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(
/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/core/module.py:410: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.pre

losses are tensor([1.1723, 1.0903, 1.1341, 0.9927, 1.1694, 1.1054],
       grad_fn=<NllLossBackward0>) and loss is 6.6641950607299805
losses are tensor([1.1051, 1.0394, 1.0429, 0.9090, 1.1205, 1.0239],
       grad_fn=<NllLossBackward0>) and loss is 6.240835189819336
losses are tensor([1.0520, 0.9976, 0.9705, 0.8443, 1.0805, 0.9591],
       grad_fn=<NllLossBackward0>) and loss is 5.904070854187012
losses are tensor([1.0086, 0.9616, 0.9113, 0.7958, 1.0473, 0.9059],
       grad_fn=<NllLossBackward0>) and loss is 5.630494117736816
losses are tensor([0.9711, 0.9298, 0.8613, 0.7551, 1.0188, 0.8627],
       grad_fn=<NllLossBackward0>) and loss is 5.398755073547363
losses are tensor([1.1798, 1.1099, 1.1089, 1.1904, 1.1312, 1.1577],
       grad_fn=<NllLossBackward0>) and loss is 6.877835750579834
losses are tensor([1.1243, 1.0320, 1.0588, 1.1286, 1.0574, 1.0879],
       grad_fn=<NllLossBackward0>) and loss is 6.4890618324279785
losses are tensor([1.0762, 0.9670, 1.0158, 1.0767, 0.9938, 1.0284],

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/core/module.py:410: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(
/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/core/module.py:410: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.pre

losses are tensor([1.0785, 1.0458, 1.0438, 0.8840, 1.1449, 1.0687],
       grad_fn=<NllLossBackward0>) and loss is 6.265613079071045
losses are tensor([0.9891, 0.9738, 0.9921, 0.8170, 1.0661, 0.9983],
       grad_fn=<NllLossBackward0>) and loss is 5.8363728523254395
losses are tensor([0.9151, 0.9096, 0.9455, 0.7611, 1.0006, 0.9364],
       grad_fn=<NllLossBackward0>) and loss is 5.468307971954346
losses are tensor([0.8516, 0.8521, 0.9024, 0.7136, 0.9450, 0.8804],
       grad_fn=<NllLossBackward0>) and loss is 5.145122528076172
losses are tensor([0.7968, 0.7996, 0.8622, 0.6732, 0.8968, 0.8290],
       grad_fn=<NllLossBackward0>) and loss is 4.857547283172607
losses are tensor([0.9713, 1.1028, 0.9567, 1.0795, 1.1544, 1.0684],
       grad_fn=<NllLossBackward0>) and loss is 6.333107948303223
losses are tensor([0.8828, 1.0492, 0.8680, 1.0370, 1.0831, 1.0072],
       grad_fn=<NllLossBackward0>) and loss is 5.927303314208984
losses are tensor([0.8073, 1.0003, 0.7919, 0.9987, 1.0201, 0.9529],


/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/core/module.py:410: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(
/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/core/module.py:410: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(
/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [5]:
from samplers.FewShotValidationEpisodeBatchSampler import FewShotValidationEpisodeBatchSampler
from samplers.FewShotValidationEpisodeSampler import FewShotValidationEpisodeSampler
from validation_datasets.ValidationDataset import ValidationDataset
import torch.utils.data as data

ds = ValidationDataset()

val_protomaml_sampler = FewShotValidationEpisodeBatchSampler(ds, kShot=2)
val_protomaml_loader = data.DataLoader(
    ds, batch_sampler=val_protomaml_sampler, collate_fn=val_protomaml_sampler.getCollateFunction(), num_workers=1
)

# val_protomaml_loader = data.DataLoader(
#     None, batch_sampler=val_protomaml_sampler, num_workers=1
# )


In [6]:
for i in range(1):
    batch = next(iter(val_protomaml_loader))
#     print(batch)
    for episode_i in range(len(batch[0])):
        data, labels = batch[0][episode_i], batch[1][episode_i]
        supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2]
        print(supportLabels)

[1, 1, 0, 0]
[2, 3, 2, 4, 4, 3]
[7, 5, 6, 5, 6, 7]
[10, 8, 8, 9, 10, 9]
[13, 12, 11, 12, 13, 11]
[15, 14, 14, 15]
[17, 16, 17, 16]
[19, 18, 18, 19]
